Add necessary imports here

In [8]:
!pip3 install pymagnitude
!wget http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude

from pymagnitude import *
vectors = Magnitude("glove.6B.300d.magnitude")

--2020-04-22 02:50:39--  http://magnitude.plasticity.ai/glove/heavy/glove.6B.300d.magnitude
Resolving magnitude.plasticity.ai (magnitude.plasticity.ai)... 52.217.9.227
Connecting to magnitude.plasticity.ai (magnitude.plasticity.ai)|52.217.9.227|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1384890368 (1.3G) [binary/octet-stream]
Saving to: ‘glove.6B.300d.magnitude’

glove.6B.300d.magni 100%[===================>]   1.29G  52.4MB/s    in 24s     

2020-04-22 02:51:04 (54.0 MB/s) - ‘glove.6B.300d.magnitude’ saved [1384890368/1384890368]



In [9]:
import numpy as np
import torch
import string
import nltk, string, os, json

from nltk import RegexpParser
from nltk.tree import Tree
import random

from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

mkdir: cannot create directory ‘encoder’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  21.4M      0  0:00:06  0:00:06 --:--:-- 25.9M


Set up models

In [41]:
# Include models.py in the directory
files.upload()
from models import InferSent
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

Saving models.py to models.py


<All keys matched successfully>

In [0]:
model.set_w2v_path('drive/Shared drives/CIS 700 Project/GloVe/glove.840B.300d.txt')

In [47]:
model.build_vocab_k_words(K=100000)

Vocab size : 100000


Cosine similarity

In [0]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [0]:
def find_similar_command(user_command, known_commands):

  sim1 = find_most_similar_command(user_command, known_commands)
  sim = []
  for k in known_commands:
    sim.append(cosine(model.encode(user_command)[0], model.encode(k)[0]))
  # sim = [float(i)/sum(sim) for i in sim]
  for i in range(len(sim)):
    sim[i] += sim1[i]
  idx = sim.index(max(sim))
  #print(sim)
  #print(idx)
  return known_commands[idx]

Add necessary dictionaries/lists and parsing methods here

In [0]:
def get_article(filename):
    with open(filename, "r") as f:
        s = [] # steps - list of tuples (method_number, step text)
        d = {} # descriptions - Description dictionary step text -> description
        m = [] # methods
        data_json = json.load(f)
        title = data_json['title']
        if title != None:
        # Get the steps
            if len(data_json['method/part']) > 0:
                methods = data_json['method/part']
                for method in methods:
                    all_steps = [(method['name'], step['headline']) for step in method['steps']]
                    s += all_steps
                    m += method['name']
                    all_descs = dict(zip([step['headline'] for step in method['steps']], [step['description'] for step in method['steps']]))
                    d.update(all_descs)      
            else:
              print(filename)
               # s = [(None, step[0]) for step in data_json['steps']]
               # d = dict(zip([step[0] for step in data_json['steps']], [step[1] for step in data_json['steps']]))
        else:
            print('Article has no title!')
        return s, d, m, title

NLTK Package Downloads

In [17]:
# download averaged_perceptron_tagger, wordnet
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Add NLTK POS-tagging and tokenizer here

In [0]:
# Source: https://stackoverflow.com/questions/29473169/approach-for-identifying-whether-a-sentence-includes-an-imperative-within-it?fbclid=IwAR1kGWHvo0M1sSMMXl4Rv4PJ-dhRchEABc83JQpAqotQnFqY3uUBD0jAMAU

def is_imperative(tagged_sent):
    # if the sentence is not a question...
    if tagged_sent[-1][0] != "?":
         #catches simple imperatives, e.g. "Open the pod bay doors, HAL!"
        
        if tagged_sent[0][1] == "VB" or tagged_sent[0][1] == "MD":
            return True
        
        # catches imperative sentences starting with words like 'please', 'you',...
        # E.g. "Dave, stop.", "Just take a stress pill and think things over."
        else:
          # check if the first word has a verb synonym
          synonyms = [syns.name() for syns in wordnet.synsets(tagged_sent[0][0])]
          # split the synonyms array into two: more related synonyms should appear earlier
          # Follow a heuristic such that if a verb synonym doesn't appear in the first half
          # then this word is likely more often used as a noun
          synonyms = synonyms[0:len(synonyms)//2]
          for synonym in synonyms:
            # if there is a verb written the same way as the first token, return true
            splitted = synonym.split('.')
            if splitted[1] == 'v':
              if splitted[0] == tagged_sent[0][0].lower():
                return True
          #chunk = get_chunks(tagged_sent)
          # check if the first chunk of the sentence is a VB-Phrase
          #if type(chunk[0]) is Tree and chunk[0].label() == "VB-Phrase":
              #return True
    # For the sake of our game, consider the questions as non-imperative  
    return False

# chunks the sentence into grammatical phrases based on its POS-tags
def get_chunks(tagged_sent):
    chunkgram = r"""VB-Phrase: {<DT><,>*<VB>}
                    VB-Phrase: {<RB><VB>}
                    VB-Phrase: {<UH><,>*<VB>}
                    VB-Phrase: {<UH><,><VBP>}
                    VB-Phrase: {<PRP><VB>}
                    VB-Phrase: {<NN.?>+<,>*<VB>}
                    Q-Tag: {<,><MD><RB>*<PRP><.>*}"""
    chunkparser = RegexpParser(chunkgram)
    return chunkparser.parse(tagged_sent)

# Tokenizer that I implemented from CIS421
def tokenize(text):
    tokens = []
    words = text.split(' ')
    for word in words:
        if len(word) == 0:
            continue
        w = []
        for c in word:
            if c in string.punctuation:
              if c == '\'':
                w.append(c)
              else:
                  if len(w) != 0:
                      tokens.append(''.join(w))
                      w = []
                  tokens.append(c)
            else:
                if c not in string.whitespace:
                    w.append(c)
        if len(w) != 0:
            tokens.append(''.join(w))
    return tokens

def imperative_check(text):
  # First tokenize the step headline
  tokens = tokenize(text)
  # Get the POS-tags
  tags = nltk.pos_tag(tokens)
  # Check if imperative
  imperative = is_imperative(tags)
  
  return imperative

Add similarity check functions here

In [0]:
#get vector for the given sentence
def construct_sentence_vector(command, vectors):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  for word in command.split():
    word_vector = vectors.query(word)
    sentence_vector += word_vector
  sentence_vector = sentence_vector / len(command.split())
  return sentence_vector

def construct_sentence_vector(command):
  sentence_vector = np.zeros(shape=(vectors.dim,))
  cnt = 0
  for word in command.split():
    word_vector = vectors.query(word)
    # TODO - Do something
    sentence_vector = sentence_vector + word_vector
    cnt+=1
  sentence_vector = sentence_vector/cnt
  return sentence_vector

#compare items in the list to the article_field. for instance, if you want to find the most similar titles
#article_field should be the title, and list should be a list of titles you want to find the most similar out of
def find_most_similar(article_field, list, vectors):
  vecs = {}
  max_sim = 0
  most_sim = list[0]
  vector_1 = construct_sentence_vector(article_field, vectors)
  for obj in list:
    vecs[obj] = construct_sentence_vector(obj, vectors)
  for obj in vecs:
    sim = vectors.similarity(vecs[obj], vector_1)
    if sim > max_sim:
      most_sim = obj
      max_sim = sim
  return most_sim

def find_most_similar_command(user_command, known_commands):
  # TODO - Do something
  user_sent = construct_sentence_vector(user_command)
  known_sent = []
  for command in known_commands:
    known_sent.append(construct_sentence_vector(command))
  #ans = vectors.similarity(user_sent, known_sent) 
  known_sent = np.array(known_sent)
  sim = vectors.similarity(user_sent, known_sent).tolist()
  # ind = sim.index(max(sim))
  # print(sim)
  # return known_commands[ind]
  return sim

#get the top k most similar
def top_k_similar(article_field, list, vectors, k):
  top = []
  for i in range(k):
    most_sim = find_most_similar(article_field, list, vectors)
    top.append(most_sim)
    list.remove(most_sim)
  return top


Add object identifier & 3rd person sentence formation here

In [0]:
# refered from https://stackoverflow.com/questions/41051125/turning-a-sentence-from-first-to-second-person

forms = {"am" : "are", "are" : "am", 'i' : 'you', 'my' : 'your', 'me' : 'you', 'mine' : 'yours', 'you' : 'I', 
'your' : 'my', 'yours' : 'mine'}

def translate(text):
  # print(text)
  result = text
  for word in text.split():
    # print(word)
    if word.lower() in forms: 
      result = result.replace(word, forms[word.lower()])
      # print(result)
  # print(result)
  return result

def changePerson(text):
  result = ""
  tags = nltk.pos_tag(tokenize(text))
  isImp = is_imperative(tags)

  if isImp:
    if text[0] == ' ' or text[0] == '\n':
      result = "You " + text[1].lower() + text[2:]
    else:
      result = "You " + text[0].lower() + text[1:]
  else:
    result = translate(text)
  return result

def identifyObjects(text):
  tags = nltk.pos_tag(tokenize(text))
  objects = []
  for t in tags:
    if t[1]=="NN":
      objects.append(t[0])
  return objects

Description Generator

In [0]:
def get_description(title):
  DATA = ""
  tit=""
  if title.split()[0].lower()=="how" and title.split()[1].lower()=="to":
    tit += "You want "
    tit += " ".join(title.split()[1:])
  # else:
  

  objects = identifyObjects(title)
  if len(objects) == 0:
    DATA = tit
  else:
    DATA += (tit + '\n')
  # print(objects)
  if len(objects)==1:
    s = "You have a " + objects[0] + "\n"
    DATA += s
  elif len(objects)>1:
    ob = ''
    ob += ', '.join(objects)
    s = "You have " + ob + '\n'
    DATA += s
  
  print(DATA)

Once imports and functions are done, upload a JSON file, get its name and parse it

In [58]:
uploaded = files.upload()

Saving 4027.json to 4027.json


In [59]:
try:
  file_name = list(uploaded.keys())[0]
  print(file_name)
except:
  f = open('/content/3029.json')
  file_name = f.name.split('/')
  file_name = file_name[len(file_name) - 1]

4027.json


In [0]:
def trim_beginning_punctuation(sentence):
  to_return = ''
  letter_seen = False
  for c in sentence:
    if c in string.punctuation or c in string.whitespace:
      if letter_seen:
        to_return += c
    else:
      if c in string.ascii_letters:
        if not letter_seen:
          letter_seen = True
        to_return += c
  return to_return

def milestone_1_results(filename, include_desc):
  # include_desc -> Do you want to include imperative description sentences as game steps?
  steps, descriptions, method_names, title = get_article(filename)
  # print the title of the article
  #print('Title: ' + title)
  print(title)
  # Once the necessary fields are filled, find imperatives
  imperatives = []

  for step in steps:
    # First split the headline into sentences
    headline_sentences = step[1].split('.')
    for headline in headline_sentences:
      trimmed_headline = trim_beginning_punctuation(headline)
      trimmed_headline = trimmed_headline.replace('\n', '')
      if len(trimmed_headline) == 0:
          continue
      if imperative_check(trimmed_headline):
        imperatives.append(trimmed_headline)
        break # No two imperatives from the same headline are added.
    if include_desc:
      description = descriptions[step[1]]
      # Split the description paragraph into sentences
      sentences = description.split('.')
      for sentence in sentences:
        trimmed_sentence = trim_beginning_punctuation(sentence)
        trimmed_sentence = trimmed_sentence.replace('\n', '')
        if len(trimmed_sentence) == 0:
          continue
        if imperative_check(trimmed_sentence):
          imperatives.append(trimmed_sentence)
          break # No two imperatives from the same description are added. !!! REMOVE THIS IF you want multiple imperatives

    
  # Once imperatives are found, print them and turn them into 3rd person
  #print('\nList of Imperatives:\n')
  #for imp in imperatives:
    #print(imp)
    
  third_person = [changePerson(imperative) for imperative in imperatives]
  return third_person, title

  # Print the 3rd person sentences in order
  #print('\nImperatives Turned Into 3rd Person:\n')
  #for third in third_person:
    #print(third)
  
  # # Find 5 similar articles to this article given the title
  # k = 5

  # # Make sure that the folder new_wikihow_1000 is in the drive - that's where we have all the data
  # file_path = '/content/drive/My Drive/Colab Notebooks/new_wikihow_1000'
  # all_files = os.listdir(file_path)
  # files_and_titles = {} # title -> filename dictionary
  # idx = 0
  # for data in all_files:
  #   if idx >= 100: # for now the number of files to read is limited to 100
  #     break
  #   with open(file_path + '/' + data, "r") as f:
  #     data_json = json.load(f)
  #     t = data_json['title']
  #     # Skip articles with no title
  #     if t == None:
  #       continue
  #     files_and_titles[t] = data
  #     idx += 1
  
  # # Once we have file titles, pass them as a list to top_k_similar()
  # k_similar = top_k_similar(title, list(files_and_titles.keys()), vectors, k)
  
  # # Print the top k similar article titles and the corresponding json file name
  # print('\nTop ' + str(k) + ' Similar Article Titles:\n')
  # for similar in k_similar:
  #   print('Title: ' + similar + ', Filename: ' + files_and_titles[similar])

Start extracting from article here:


In [60]:
# Send True to include descriptions as well!
known_commands, title = milestone_1_results(file_name, False)

How to Make a Beaded Necklace


In [61]:
try:
  embeddings = model.encode(known_commands, bsize=128, tokenize=False, verbose=True)
  print('nb sentences encoded : {0}'.format(len(embeddings)))
except:
  print('This article has no imperatives!')

Nb words kept : 103/109 (94.5%)
Speed : 22.0 sentences/s (cpu mode, bsize=128)
nb sentences encoded : 10


Method for choice picking

In [0]:
def pick_choices(current_step, commands):
  if len(commands) - 1 == current_step:
    return None, None
  elif len(commands) - 2 == current_step:
    # Can only pick one wrong choice
    return len(commands) - 1, None
  else:
    # pick 2 that are not in steps_seen
    options_first = [num for num in range(current_step + 1, len(commands))]
    first = random.choice(options_first)
    options_second = [num for num in range(current_step + 1, len(commands)) if num != first]
    second = random.choice(options_second)
    return first,second

Start the game here:

In [62]:
current_step = 0
steps_seen = []
end_game = False
get_description(title)
while not end_game:
  if current_step == len(known_commands):
    end_game = True
    print('You reached the end of the game')
  else:
    current_step_text = known_commands[current_step]
    # get the description of current step text
    get_description(current_step_text)
    print('What do you do?')
    print('You:')
    first, second = pick_choices(current_step, known_commands)
    choices = []
    # if wrong choices exist
    if first:
      if second:
        choices.append(known_commands[second])
      choices.append(known_commands[first])
    choices.append(known_commands[current_step])
    correct_choice = known_commands[current_step] # this is the correct choice
    random.shuffle(choices) # shuffle the order of choices
    for choice in choices:
      print('\t' + choice[4:])
    correct_found = False
    while not correct_found:
      user_pick = input('>')
      if user_pick.lower() == 'quit':
        correct_found = True
        end_game = True
        print('You ended the game.')
        continue
      # Check if what the user picked is correct
      most_similar = find_similar_command(user_pick, known_commands)
      if most_similar[4:].lower() == correct_choice[4:].lower():
        # Correct!
        current_step += 1
        correct_found = True
      else:
        print('Try again.')


You want to Make a Beaded Necklace
You have a Necklace


You have a beading

What do you do?
You:
	gather  crimp beads,  clasp, and the beads for the desired necklace
	gather your beading materials
	thread the end of the stringing material through the clasp section
>gather beading materials

You have necklace, style

What do you do?
You:
	select a length
	place your bead board on your flat surface
	determine your necklace style
>determine your necklace style

You have a length

What do you do?
You:
	use a clasp section\/ jump ring and the bead-crimp-bead combo
	arrange your beads on a flat surface, like a table or a desk
	select a length
>quit
You ended the game.
